### Preamble

#### Project Template

In [ ]:
%load_ext autoreload

In [ ]:
import os as _os
_os.chdir(_os.environ['PROJECT_ROOT'])
_os.path.realpath(_os.path.curdir)

#### Imports

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
from lib.pandas_util import idxwhere, aligned_index, align_indexes, invert_mapping
import matplotlib as mpl
import lib.plot
import statsmodels.api as sm
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm
import subprocess
from tempfile import mkstemp
import time
import subprocess
from itertools import chain
import os
from itertools import product
from mpl_toolkits.axes_grid1 import make_axes_locatable
from datetime import datetime
import sys
from glob import glob

In [ ]:
import lib.thisproject.data

#### Set Style

In [ ]:
sns.set_context('talk')
plt.rcParams['figure.dpi'] = 50

## Load Data

In [ ]:
mwas = []

for path in glob('data/group/xjin_hmp2/species/sp-*/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr100-depth250.strain_mwas.tsv'):
    species = path[len('data/group/xjin_hmp2/species/sp-'):-len('/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr100-depth250.strain_mwas.tsv')]
    d = pd.read_table(path)
    mwas.append(d.assign(species=species))
mwas = pd.concat(mwas)

In [ ]:
len(mwas.gene.unique())

In [ ]:
mwas.sort_values('logratio_pc')#.head(20)

In [ ]:
bins = np.logspace(-5, 0, num=20)
# bins = np.linspace(0, 1, num=51)
for ibd_diagnosis in mwas.ibd_diagnosis.unique():
    plt.hist(mwas[lambda x: x.ibd_diagnosis == ibd_diagnosis].pvalue, label=ibd_diagnosis, alpha=0.5, bins=bins)

plt.plot(bins[1:], (bins[1:] - bins[:-1]) * mwas.shape[0] / 3)
plt.legend()

plt.xscale('log')
plt.yscale('log')

In [ ]:
mwas.assign(significant=lambda x: x.pvalue < 1e-3).significant.mean()

In [ ]:
fraction_significant = mwas.assign(significant=lambda x: x.pvalue < 1e-3).groupby(['species', 'ibd_diagnosis']).significant.mean().sort_values(ascending=False)

In [ ]:
fraction_significant.xs('nonIBD', level='ibd_diagnosis')

In [ ]:
fraction_significant.xs('CD', level='ibd_diagnosis')

In [ ]:
fraction_significant.xs('UC', level='ibd_diagnosis')

In [ ]:
num_significant_pvalues = mwas[lambda x: x.pvalue < 1e-2].groupby('species').pvalue.count()
total_num_tests = mwas.groupby('species').pvalue.count()

(num_significant_pvalues / total_num_tests).fillna(0).sort_values(ascending=False).head(10)

In [ ]:
plt.scatter(
    'log_logratio_pc',
    'pvalue',
    data=mwas.assign(log_logratio_pc=lambda x: np.log2(x.logratio_pc)),
)
plt.yscale('log')
# plt.xscale('log')
plt.gca().invert_yaxis()

In [ ]:
d0 = mwas[mwas.ibd_diagnosis == 'UC'].assign(log_logratio_pc=lambda x: np.log2(x.logratio_pc))

plt.hist2d(
    'log_logratio_pc',
    'pvalue',
    data=d0,
    bins=(np.linspace(-10, 10, num=100), np.logspace(-6, 0, num=100)),
    norm=mpl.colors.SymLogNorm(linthresh=2, linscale=3),
    cmin=1,
    cmap='copper_r',
)
plt.colorbar()
plt.yscale('log')
# plt.xscale('log')
plt.gca().invert_yaxis()
plt.xlabel('Log(Odds Ratio)')
plt.ylabel('P-value')

d1 = d0[lambda x: (x.gene == 'UHGG000638_01150') & (x.pvalue < 1e-3)]
for _, d2 in d1.iterrows():
    plt.annotate('', xy=(d2.log_logratio_pc, d2.pvalue), ha='left', va='bottom', xytext=(10, 10), textcoords="offset points", arrowprops=dict())
d1

In [ ]:
d1.squeeze().Description

In [ ]:
d0.sort_values('pvalue').head(10)